In [94]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import requests

In [101]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# ✅ Configuration Selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Désactiver pour voir l'exécution
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36")

# ✅ Initialiser WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ Ouvrir la page Fragrantica
url = "https://www.fragrantica.fr/search/"
driver.get(url)

# ✅ Attendre le chargement initial
time.sleep(5)

# ✅ Boucle pour cliquer sur "Voir plus de résultats"
max_clicks = 50  # Sécurité pour éviter les boucles infinies
clicks = 0

while clicks < max_clicks:
    try:
        # ✅ Vérifier si le bouton "Voir plus de résultats" est présent
        afficher_plus_buttons = driver.find_elements(By.XPATH, "//button[contains(text(), 'Voir plus de résultats')]")
        
        if len(afficher_plus_buttons) == 0:
            print("🚨 Aucun bouton 'Voir plus de résultats' trouvé. Fin du scrolling.")
            break  # Sortir de la boucle

        # ✅ Prendre le dernier bouton trouvé (au cas où il y en a plusieurs)
        afficher_plus_button = afficher_plus_buttons[-1]
        
        # ✅ Scroller jusqu'au bouton pour s'assurer qu'il est visible
        driver.execute_script("arguments[0].scrollIntoView();", afficher_plus_button)
        time.sleep(random.uniform(1, 2))  # Pause aléatoire pour éviter d’être détecté

        # ✅ Clic avec JavaScript si le clic normal échoue
        driver.execute_script("arguments[0].click();", afficher_plus_button)

        print(f"✅ Clique {clicks+1} sur 'Voir plus de résultats'")
        clicks += 1

        # ✅ Attendre que de nouveaux éléments apparaissent
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "card-section"))
        )
        
        # ✅ Vérifier si de nouveaux éléments sont apparus
        elements_before = len(driver.find_elements(By.CLASS_NAME, "card-section"))
        time.sleep(random.uniform(2, 4))  # Attendre un peu
        elements_after = len(driver.find_elements(By.CLASS_NAME, "card-section"))

        if elements_before == elements_after:
            print("🚨 Aucun nouvel élément détecté. Fin du scrolling.")
            break  # Sortie de boucle si rien ne change

    except Exception as e:
        print(f"🚨 Erreur détectée : {e}")
        break  # Sortir si une erreur majeure survient
    
# Récupérer le HTML après exécution de JavaScript
html = driver.page_source
driver.quit()

soup = BeautifulSoup(html, "html.parser")
parfums = soup.find_all("div", class_="card-section")

data = []
for parfum in parfums:
    a_tag = parfum.find("a")
    if a_tag:
        lien=a_tag["href"]
        nom = a_tag.text.strip()
        data.append({"Nom": nom, "Lien":lien})

df = pd.DataFrame(data)
df.to_csv("parfums_fragrantica.csv", index=False)

✅ Clique 1 sur 'Voir plus de résultats'
✅ Clique 2 sur 'Voir plus de résultats'
✅ Clique 3 sur 'Voir plus de résultats'
✅ Clique 4 sur 'Voir plus de résultats'
✅ Clique 5 sur 'Voir plus de résultats'
✅ Clique 6 sur 'Voir plus de résultats'
✅ Clique 7 sur 'Voir plus de résultats'
✅ Clique 8 sur 'Voir plus de résultats'
✅ Clique 9 sur 'Voir plus de résultats'
✅ Clique 10 sur 'Voir plus de résultats'
✅ Clique 11 sur 'Voir plus de résultats'
✅ Clique 12 sur 'Voir plus de résultats'
✅ Clique 13 sur 'Voir plus de résultats'
✅ Clique 14 sur 'Voir plus de résultats'
✅ Clique 15 sur 'Voir plus de résultats'
✅ Clique 16 sur 'Voir plus de résultats'
✅ Clique 17 sur 'Voir plus de résultats'
✅ Clique 18 sur 'Voir plus de résultats'
✅ Clique 19 sur 'Voir plus de résultats'
✅ Clique 20 sur 'Voir plus de résultats'
✅ Clique 21 sur 'Voir plus de résultats'
✅ Clique 22 sur 'Voir plus de résultats'
✅ Clique 23 sur 'Voir plus de résultats'
✅ Clique 24 sur 'Voir plus de résultats'
✅ Clique 25 sur 'Voir plu

In [102]:
data = pd.read_csv('parfums_fragrantica.csv')
data.head()

,Nom,Lien
0,Khamrah,https://www.fragrantica.fr/parfum/Lattafa-Perf...
1,Le Male Elixir,https://www.fragrantica.fr/parfum/Jean-Paul-Ga...
2,Angels' Share,https://www.fragrantica.fr/parfum/By-Kilian/An...
3,Le Male Le Parfum,https://www.fragrantica.fr/parfum/Jean-Paul-Ga...
4,XJ 1861 Naxos,https://www.fragrantica.fr/parfum/Xerjoff/XJ-1...


In [103]:
data.shape

(1000, 2)